# DMRG

In this notebook we will study the ground state of a spin Hamiltonian
$$H = \sum_{i=1}^N \sigma^z_i \sigma^z_i + h \sum_i \sigma^x_i$$
using matrix product states and the density matrix renormalization group algorithm (DMRG in short).

First, we assume the SeeMPS library has been installed in the environment, and import a set of tools.

In [ ]:
from seemps.hamiltonians import ConstantTIHamiltonian
from seemps.optimization import dmrg
from seemps.state import MPS, product_state
import numpy as np

Next, we define our operators. The `ConstantTIHamiltonian` class is a tool to construct matrix product operators for a given problem. It allows you to define the interaction between nearest-neighbor sites and some local terms, and move ahead.

In [ ]:
N = 10
h = 0.2
sx = np.array([[0, 1], [1, 0]])
sz = np.array([[1, 0], [0, -1]])
H = ConstantTIHamiltonian(size=N, interaction=np.kron(sz, sz), local_term=h * sx)

We use the DMRG algorithm with a suitable guess to find out the ground state energy

In [ ]:
result = dmrg(H, guess=product_state(np.ones(2) / np.sqrt(2), N))
print(f"DMRG energy = {result.energy}")

We can compare with the exact diagonalization, because the problem is small

In [ ]:
from scipy.sparse.linalg import eigsh

E, _ = eigsh(H.to_matrix(), k=1, which="SA")
print(f"Exact energy = {result.energy}")

With this, we can do a slightly more sophisticated study of the quantum phase transition.

In [ ]:
import matplotlib.pyplot as plt
from time import process_time


def experiment_00(N: int = 30, steps: int = 21):
    """Explore the quantum phase transition as a function of the transverse
    magnetic field $h$. Plots the magnetizations of the ground state, computed
    with a small symmetry breaking term."""
    h = np.linspace(0, 2, steps)
    epsilon = 0
    sx = np.array([[0, 1], [1, 0]])
    sz = np.array([[1, 0], [0, -1]])
    sz_values = []
    sx_values = []
    extras = ""

    ZZ = ConstantTIHamiltonian(size=N, interaction=np.kron(sz, sz))
    X = ConstantTIHamiltonian(size=N, local_term=sx)
    for hi in h:
        H = ConstantTIHamiltonian(
            size=N, interaction=np.kron(sz, sz), local_term=-hi * sx + epsilon * sz
        )
        time = process_time()
        result = dmrg(H, guess=product_state(np.ones(2) / np.sqrt(2), N), maxiter=100)
        time = process_time() - time
        sz_values.append(ZZ.to_mpo().expectation(result.state))
        sx_values.append(X.to_mpo().expectation(result.state))

        if N <= 20:
            E, psi0 = eigsh(H.to_matrix(), v0=result.state.to_vector(), k=1, which="SA")
            E = E[0]
            psi0 = psi0[:, 0]
            sz_exact = np.vdot(psi0, ZZ.to_matrix() @ psi0)
            sx_exact = np.vdot(psi0, X.to_matrix() @ psi0)
            extras = (
                f", errors: E = {abs(E - result.energy):6e}, "
                f"Sz = {sz_exact - sz_values[-1]:6e}, "
                f"Sx = {sx_exact - sx_values[-1]:6e}"
            )
        print(f"Energy = {result.energy:+6f}, time = {time:5f}s" + extras)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4))
    ax1.plot(h, sz_values)
    ax1.set_xlabel(r"$h$")
    ax1.set_ylabel(r"$\langle{\sigma^z_{i}\cdot \sigma^z_{i+1}}\rangle$")
    ax2.plot(h, sx_values)
    ax2.set_xlabel(r"$h$")
    ax2.set_ylabel(r"$\langle{S_x}\rangle$")
    fig.tight_layout()


experiment_00(N=15)

In [ ]:
experiment_00(N=30, steps=31)